In [1]:
case class User(userId: Long, userName: String)

case class UserActivity(userId: Long, activityTypeId: Int, timestampEpochSec: Long)

val LoginActivityTypeId = 0
val LogoutActivityTypeId = 1

Intitializing Scala interpreter ...

Spark Web UI available at http://admins-mbp.lan:4043
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1607429533340)
SparkSession available as 'spark'


defined class User
defined class UserActivity
LoginActivityTypeId: Int = 0
LogoutActivityTypeId: Int = 1


In [13]:
import org.apache.spark.rdd.RDD

def readUserData(): RDD[User] = {
 sc.parallelize(
   Array(
     User(1, "Doe, John"),
     User(2, "Doe, Jane"),
     User(3, "X, Mr."))
 )
}

import org.apache.spark.rdd.RDD
readUserData: ()org.apache.spark.rdd.RDD[User]


In [14]:
def readUserActivityData(): RDD[UserActivity] = {
 sc.parallelize(
   Array(
     UserActivity(1, LoginActivityTypeId, 1514764800L),
     UserActivity(2, LoginActivityTypeId, 1514808000L),
     UserActivity(1, LogoutActivityTypeId, 1514829600L),
     UserActivity(1, LoginActivityTypeId, 1514894400L))
 )
}

readUserActivityData: ()org.apache.spark.rdd.RDD[UserActivity]


In [26]:
def calculate(): Unit = {
 val userRdd: RDD[(Long, User)] =
   readUserData().map(e => (e.userId, e))
 val userActivityRdd: RDD[(Long, UserActivity)] =
   readUserActivityData().map(e => (e.userId, e))
 
 val result1 = userRdd
   .leftOuterJoin(userActivityRdd) //for debugging purpose

 val result = userRdd
   .leftOuterJoin(userActivityRdd)
   .filter(e => e._2._2.isDefined && e._2._2.get.activityTypeId == LoginActivityTypeId)
   .map(e => (e._2._1.userName, e._2._2.get.timestampEpochSec))
   .reduceByKey((a, b) => if (a < b) a else b)

 result1
   .foreach(println)    //for debugging purpose

 result
   .foreach(e => println(s"${e._1}: ${e._2}"))

}

calculate: ()Unit


In [27]:
calculate()

Doe, Jane: 1514808000
Doe, John: 1514764800
(3,(User(3,X, Mr.),None))
(1,(User(1,Doe, John),Some(UserActivity(1,0,1514764800))))
(1,(User(1,Doe, John),Some(UserActivity(1,1,1514829600))))
(1,(User(1,Doe, John),Some(UserActivity(1,0,1514894400))))
(2,(User(2,Doe, Jane),Some(UserActivity(2,0,1514808000))))


<img src="images/partition_tuning.png">

"